## SQL Agent
The third method I've seen on langchain is to use the SQL Database agent. It is designed to be able to handle more general questions and recover errors.

The docs say it's possible it may perform some DML statements on the database, so I won't feed it any prompts targeted at changing the data. This will be my last try before moving onto more custom methods.

In [10]:
from watermark import watermark
print(watermark())

Last updated: 2023-08-01T00:53:06.219492-07:00

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.14.0

Compiler    : Clang 15.0.7 
OS          : Darwin
Release     : 22.5.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



### Imports

In [1]:
import os
from dotenv import load_dotenv

from langchain import HuggingFaceHub
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

### Point to db
This time I'll just cut it short and point to the one.

In [3]:
db = SQLDatabase.from_uri("sqlite:///" + "/Users/brettly/Sboard/projects/text-to-sql/data/processed/db/department_management.sqlite")

### Setup LLM & Toolkit

In [4]:
load_dotenv()
hf_api_token = os.getenv('hf_token')

repo = 'google/flan-t5-xxl'

llm = HuggingFaceHub(repo_id=repo, huggingfacehub_api_token=hf_api_token, model_kwargs={"temperature": 0.2, "max_length": 1500})

In [7]:
#agents use toolkits, and I'll use the standard pre-defined one
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

### Initialize the Agent Using Zero Shot React Description

In [8]:
#additional things to feed into the "agent_executor" instead of a chain
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
)

https://github.com/hwchase17/langchain/issues/4293

### Test - Describe Table

In [9]:
agent_executor("How many rows are in the 'head' table of the 'department_management' schema?")



> Entering new  chain...


OutputParserException: Could not parse LLM output: `sql_db_schema head - schema department_management n = `

I'm not actually able to successfully run this in the manner the documentation suggests. It gives me an error saying it can't parse the Action the model is looking to do. As I see this in simple terms, it almost appears the model has decided what it wants to do, but never actually performs that action on the database. The challenge I have is that I'm not sure if that is a model issue, prompt issue, or agent bug. Or even if the agent is compatible with non-OpenAI models!

In doing some research, it sounds like it's likely token count restrictions. So this may work with an Open AI or local model, but not through the Inference API. 

I want to do as much of this as possible in a no-cost way, so I'll go back to the drawing board and look to take a more custom approach.